In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q transformers datasets accelerate evaluate scikit-learn wandb
!pip install -q torch torchvision --extra-index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [3]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import torch
import os
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, DataCollatorWithPadding
from torch.utils.data import DataLoader
from sklearn.utils import resample

In [4]:
# Load Dataset
data_path = Path("/content/drive/MyDrive/deteksi_cd-indo/preprocessed.csv")
df = pd.read_csv(data_path)

print("Rows:", len(df))
print("Columns:", df.columns.tolist())
# contoh 5 baris
display(df.head())

Rows: 7484
Columns: ['id', 'text', 'label', 'text_clean', 'text_stem', 'tokens', 'label_id']


,id,text,label,text_clean,text_stem,tokens,label_id
0,1,ketika saya sudah berusaha semaksimal mungkin ...,All-or-nothing,ketika saya sudah berusaha semaksimal mungkin ...,ketika saya sudah usaha maksimal mungkin tapi ...,"['ketika', 'saya', 'sudah', 'usaha', 'maksimal...",0
1,2,Saya ketika SD hingga SMP dulu beranggapan bah...,All-or-nothing,saya ketika sd hingga smp dulu beranggapan bah...,saya ketika sd hingga smp dulu anggap bahwa ke...,"['saya', 'ketika', 'sd', 'hingga', 'smp', 'dul...",0
2,3,Saya berfikir jika tidak lolos MBKM maka masa ...,All-or-nothing,saya berfikir jika tidak lolos mbkm maka masa ...,saya berfikir jika tidak lolos mbkm maka masa ...,"['saya', 'berfikir', 'jika', 'tidak', 'lolos',...",0
3,4,"Di desa saya dulu ada dua kelompok pertemanan,...",All-or-nothing,di desa saya dulu ada dua kelompok pertemanan ...,di desa saya dulu ada dua kelompok teman nah s...,"['di', 'desa', 'saya', 'dulu', 'ada', 'dua', '...",0
4,5,Saya hanya berteman dengan orang yang tidak me...,All-or-nothing,saya hanya berteman dengan orang yang tidak me...,saya hanya teman dengan orang yang tidak sakit...,"['saya', 'hanya', 'teman', 'dengan', 'orang', ...",0


In [5]:
# Split dataset (80% train,10% val, 10% test)
train_df, temp_df = train_test_split(
    df, test_size=0.2,
    stratify=df["label"],
    random_state=42
)
val_df, test_df = train_test_split(
    temp_df, test_size=0.5,
    stratify=temp_df["label"],
    random_state=42
)

In [6]:
print("=== TRAIN DATA ===")
print(len(train_df))
display(train_df.head())  # menampilkan 5 baris pertama

print("\n=== VALIDASI DATA ===")
print(len(val_df))
display(val_df.head())

print("\n=== TEST DATA ===")
print(len(test_df))
display(test_df.head())

=== TRAIN DATA ===
5987


,id,text,label,text_clean,text_stem,tokens,label_id
173,174,"Saya mendapat nilai memuaskan, namun Ia tidak ...",Discounting the positives,saya mendapat nilai memuaskan namun ia tidak m...,saya dapat nilai muas namun ia tidak rasa itu ...,"['saya', 'dapat', 'nilai', 'muas', 'namun', 'i...",1
1180,1181,Saya pernah berpikir bahwa $saya sulit mendapa...,Jumping to Conclusions,saya pernah berpikir bahwa saya sulit mendapat...,saya pernah pikir bahwa saya sulit dapat teman...,"['saya', 'pernah', 'pikir', 'bahwa', 'saya', '...",3
2419,2422,"Pernah suatu kali, saya menghadapi situasi di ...",No Distortion,pernah suatu kali saya menghadapi situasi di m...,pernah suatu kali saya hadap situasi di mana s...,"['pernah', 'suatu', 'kali', 'saya', 'hadap', '...",7
6980,7019,Dari seorang remaja di AS: Orang tua saya tela...,No Distortion,dari seorang remaja di as orang tua saya telah...,dari orang remaja di as orang tua saya telah c...,"['dari', 'orang', 'remaja', 'di', 'as', 'orang...",7
176,177,sering terjadi apabila kita menganggap remeh s...,Discounting the positives,sering terjadi apabila kita menganggap remeh s...,sering jadi apabila kita anggap remeh suatu to...,"['sering', 'jadi', 'apabila', 'kita', 'anggap'...",1



=== VALIDASI DATA ===
748


,id,text,label,text_clean,text_stem,tokens,label_id
2394,2397,"Saya memiliki nilai buruk, $saya merasa seharu...",Should statement,saya memiliki nilai buruk saya merasa seharusn...,saya milik nilai buruk saya rasa harus saya aj...,"['saya', 'milik', 'nilai', 'buruk', 'saya', 'r...",10
6123,6161,Orang yang dicintai menderita mutisme karena m...,No Distortion,orang yang dicintai menderita mutisme karena m...,orang yang cinta derita mutisme karena alami p...,"['orang', 'yang', 'cinta', 'derita', 'mutisme'...",7
2819,2823,Saya pernah menyukai seorang kakak kelas yang ...,No Distortion,saya pernah menyukai seorang kakak kelas yang ...,saya pernah suka orang kakak kelas yang mana s...,"['saya', 'pernah', 'suka', 'orang', 'kakak', '...",7
6761,6800,Kami telah berkencan selama 8 bulan sekarang.K...,Magnification or Minimization,kami telah berkencan selama 8 bulan sekarang k...,kami telah kencan lama 8 bulan sekarang dua mu...,"['kami', 'telah', 'kencan', 'lama', '8', 'bula...",5
461,462,saya pernah berbicara mungkin $saya tidak akan...,Jumping to Conclusions,saya pernah berbicara mungkin saya tidak akan ...,saya pernah bicara mungkin saya tidak akan per...,"['saya', 'pernah', 'bicara', 'mungkin', 'saya'...",3



=== TEST DATA ===
749


,id,text,label,text_clean,text_stem,tokens,label_id
2324,2327,"Dahulu ketika tes masuk politeknik impian, $Se...",Should statement,dahulu ketika tes masuk politeknik impian seha...,dahulu ketika tes masuk politeknik impi harus ...,"['dahulu', 'ketika', 'tes', 'masuk', 'politekn...",10
3309,3314,Hal ini biasa terjadi saat berinteraksi dengan...,No Distortion,hal ini biasa terjadi saat berinteraksi dengan...,hal ini biasa jadi saat interaksi dengan orang...,"['hal', 'ini', 'biasa', 'jadi', 'saat', 'inter...",7
6631,6670,"Untuk sementara waktu, teman dan tetangga saya...",No Distortion,untuk sementara waktu teman dan tetangga saya ...,untuk sementara waktu teman dan tetangga saya ...,"['untuk', 'sementara', 'waktu', 'teman', 'dan'...",7
4996,5033,Saya telah menikah selama 15 tahun.Saya telah ...,Labeling,saya telah menikah selama 15 tahun saya telah ...,saya telah meni lama 15 tahun saya telah kemba...,"['saya', 'telah', 'meni', 'lama', '15', 'tahun...",4
5716,5754,Masalah ini sudah dimulai sejak lama: apa pun ...,Jumping to Conclusions,masalah ini sudah dimulai sejak lama apa pun y...,masalah ini sudah mulai sejak lama apa pun yan...,"['masalah', 'ini', 'sudah', 'mulai', 'sejak', ...",3


In [7]:
print(train_df['label'].value_counts())
print(val_df['label'].value_counts())
print(test_df['label'].value_counts())

label
No Distortion                    2524
Jumping to Conclusions            661
Labeling                          444
Should statement                  381
Overgeneralization                361
Personalization and Blame         349
Magnification or Minimization     310
Emotional Reasoning               266
Mental filter                     258
All-or-nothing                    240
Discounting the positives         193
Name: count, dtype: int64
label
No Distortion                    315
Jumping to Conclusions            83
Labeling                          55
Should statement                  48
Overgeneralization                45
Personalization and Blame         44
Magnification or Minimization     39
Emotional Reasoning               33
Mental filter                     32
All-or-nothing                    30
Discounting the positives         24
Name: count, dtype: int64
label
No Distortion                    316
Jumping to Conclusions            83
Labeling                       

In [8]:
# simpan hasil split
out_dir = '/content/drive/MyDrive/deteksi_cd-indo'

# Simpan setiap subset
train_df.to_csv('/content/drive/MyDrive/deteksi_cd-indo/train.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/deteksi_cd-indo/val.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/deteksi_cd-indo/test.csv', index=False)

print("Dataset berhasil disimpan ke:")
print(out_dir)

Dataset berhasil disimpan ke:
/content/drive/MyDrive/deteksi_cd-indo


In [9]:
# Tokenisasi
# Konfigurasi utama
model_name = "indobenchmark/indobert-base-p2"

text_col = "text"      # kolom teks
label_col = "label"    # kolom label string
label_id_col = "label_id"  # kolom label numeric
max_lenght = 128
batch_size = 16
num_workers = 2
save_tokenized_dir = "/content/drive/MyDrive/deteksi_cd-indo"  # tempat menyimpan hasil tokenized

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

Using device: cpu


In [10]:
# Pastikan kolom label_id ada dan integer
for df in (train_df, val_df, test_df):
    if label_id_col not in df.columns:
        # buat mapping dari label string jika belum ada
        labels_sorted = sorted(pd.concat([train_df[label_col], val_df[label_col], test_df[label_col]]).unique())
        label2id = {lab: i for i, lab in enumerate(labels_sorted)}
        df["label_id"] = df[label_col].map(label2id)
    else:
        df["label_id"] = df[label_id_col].astype(int)

# (Optional) Tampilkan ukuran
print("Sizes -> train:", len(train_df), "val:", len(val_df), "test:", len(test_df))

Sizes -> train: 5987 val: 748 test: 749


In [11]:
# Buat HuggingFace Dataset dari tiap df
hf_train = Dataset.from_pandas(train_df[[text_col, "label_id"]].rename(columns={"label_id":"labels"}))
hf_val   = Dataset.from_pandas(val_df[[text_col, "label_id"]].rename(columns={"label_id":"labels"})).remove_columns('__index_level_0__')
hf_test  = Dataset.from_pandas(test_df[[text_col, "label_id"]].rename(columns={"label_id":"labels"})).remove_columns('__index_level_0__')

display(hf_train)
display(hf_val)
display(hf_test)

Dataset({
    features: ['text', 'labels', '__index_level_0__'],
    num_rows: 5987
})

Dataset({
    features: ['text', 'labels'],
    num_rows: 748
})

Dataset({
    features: ['text', 'labels'],
    num_rows: 749
})

In [12]:
# Fungsi tokenisasi
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

def tokenize_batch(batch):
    # 'batch' adalah dict, kembalikan dict yang mengandung input_ids, attention_mask
    return tokenizer(batch[text_col],
                     truncation=True,
                     max_length=max_lenght,
                     padding=False)  # jangan padding di sini, kita pakai DataCollator

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
# Tokenisasi batched
hf_train_tok = hf_train.map(tokenize_batch, batched=True, remove_columns=[text_col])
hf_val_tok   = hf_val.map(tokenize_batch, batched=True, remove_columns=[text_col])
hf_test_tok  = hf_test.map(tokenize_batch, batched=True, remove_columns=[text_col])

# Simpan tokenized dataset supaya tidak perlu tokenisasi ulang
os.makedirs(save_tokenized_dir, exist_ok=True)
hf_train_tok.save_to_disk(os.path.join(save_tokenized_dir, "train"))
hf_val_tok.save_to_disk(os.path.join(save_tokenized_dir, "validation"))
hf_test_tok.save_to_disk(os.path.join(save_tokenized_dir, "test"))
print("Tokenized datasets saved to:", save_tokenized_dir)

Map:   0%|          | 0/5987 [00:00<?, ? examples/s]

Map:   0%|          | 0/748 [00:00<?, ? examples/s]

Map:   0%|          | 0/749 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5987 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/748 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/749 [00:00<?, ? examples/s]

Tokenized datasets saved to: /content/drive/MyDrive/deteksi_cd-indo


In [14]:
# Data collator & DataLoader (PyTorch)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding="longest")  # dynamic padding

# Convert to PyTorch format, buat DataLoader
train_loader = DataLoader(hf_train_tok.with_format("torch"), batch_size=batch_size, shuffle=True,
                          collate_fn=data_collator, num_workers=num_workers)
val_loader   = DataLoader(hf_val_tok.with_format("torch"), batch_size=batch_size, shuffle=False,
                          collate_fn=data_collator, num_workers=num_workers)
test_loader  = DataLoader(hf_test_tok.with_format("torch"), batch_size=batch_size, shuffle=False,
                          collate_fn=data_collator, num_workers=num_workers)

In [15]:
# Cek satu batch bentuknya
import torch
batch = next(iter(train_loader))
print("Batch keys:", list(batch.keys()))
print("input_ids shape:", batch["input_ids"].shape)
print("attention_mask shape:", batch["attention_mask"].shape)
print("labels shape:", batch["labels"].shape)

Batch keys: ['labels', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']
input_ids shape: torch.Size([16, 128])
attention_mask shape: torch.Size([16, 128])
labels shape: torch.Size([16])
